In [21]:
### 计算遗漏值 ###

import pandas as pd
# 超参数
COLS_BALL_1 = '开奖号码1'
COLS_BALL_2 = '开奖号码2'
COLS_BALL_3 = '开奖号码3'
COLS_ALL = ['期号', '开奖日期', '开奖号码', COLS_BALL_1, COLS_BALL_2, COLS_BALL_3]
file_path = 'Data\福彩3D历史开奖.csv'
file_path_save = 'Data\福彩3D历史开奖_间隔.csv'

# 读取excel文件并获取指定列
df = pd.read_csv(file_path, usecols=COLS_ALL, encoding='GBK')

# 如果遇到错误，可以打印文件的列名来检查
if df.empty:
    print("列名可能不匹配，这里是文件的列名：")
    with open(file_path, 'r') as f:
        print(f.readline().strip().split(','))  # 假设CSV文件使用逗号分隔

print(df.head(10))


        期号           开奖日期  开奖号码1  开奖号码2  开奖号码3  开奖号码
0  2024144  2024-06-02(日)      8      1      6   816
1  2024143  2024-06-01(六)      9      5      5   955
2  2024142  2024-05-31(五)      8      1      3   813
3  2024141  2024-05-30(四)      2      5      6   256
4  2024140  2024-05-29(三)      5      3      6   536
5  2024139  2024-05-28(二)      7      4      8   748
6  2024138  2024-05-27(一)      9      5      5   955
7  2024137  2024-05-26(日)      9      8      2   982
8  2024136  2024-05-25(六)      2      2      7   227
9  2024135  2024-05-24(五)      0      3      5    35


In [22]:
## Method 1: 计算整个号码的遗漏值

# 创建一个空字典来存储每个号码上一次出现的索引
last_seen_index = {}

# 初始化一个新列来存储间隔
df['间隔'] = None

# 倒序遍历DataFrame，更新字典中的索引，并计算间隔
for current_index in range(len(df) - 1, -1, -1):
    number = df.at[current_index, '开奖号码']
    if number in last_seen_index:
        # 如果当前号码之前出现过，计算间隔
        interval = last_seen_index[number] - current_index
        df.at[current_index, '间隔'] = interval
    # 更新或添加号码的上一次出现索引
    last_seen_index[number] = current_index
    
# 打印结果
print(df)

#

           期号           开奖日期  开奖号码1  开奖号码2  开奖号码3  开奖号码    间隔
0     2024144  2024-06-02(日)      8      1      6   816   826
1     2024143  2024-06-01(六)      9      5      5   955     5
2     2024142  2024-05-31(五)      8      1      3   813   990
3     2024141  2024-05-30(四)      2      5      6   256   716
4     2024140  2024-05-29(三)      5      3      6   536   687
...       ...            ...    ...    ...    ...   ...   ...
3925  2013038  2013-02-07(四)      6      4      0   640  None
3926  2013037  2013-02-06(三)      3      6      4   364  None
3927  2013036  2013-02-05(二)      6      5      8   658  None
3928  2013033  2013-02-02(六)      2      7      7   277  None
3929  2013030  2013-01-30(三)      7      3      2   732  None

[3930 rows x 7 columns]


In [23]:
## Method 2: 找单球遗漏

# 找遗漏：找到列中每个元素之后相同数字出现的下一个索引与当前索引的间隔
def find_next_occurrence(column, value, start_index):
    for i in range(start_index + 1, len(column)):
        if column[i] == value:
            yilou = i - start_index - 1
            return yilou 
    return -1

# 应用这个函数到 DataFrame 的列 'A' 上
COLS_BALL_1_YiLou = '1号球遗漏'
COLS_BALL_2_YiLou = '2号球遗漏'
COLS_BALL_3_YiLou = '3号球遗漏'
df[COLS_BALL_1_YiLou] = df.apply(lambda row: find_next_occurrence(df[COLS_BALL_1], row[COLS_BALL_1], df.index.get_loc(row.name)), axis=1)
df[COLS_BALL_2_YiLou] = df.apply(lambda row: find_next_occurrence(df[COLS_BALL_2], row[COLS_BALL_2], df.index.get_loc(row.name)), axis=1)
df[COLS_BALL_3_YiLou] = df.apply(lambda row: find_next_occurrence(df[COLS_BALL_3], row[COLS_BALL_3], df.index.get_loc(row.name)), axis=1)

# 打印结果
print(df)



           期号           开奖日期  开奖号码1  开奖号码2  开奖号码3  开奖号码    间隔  1号球遗漏  2号球遗漏  \
0     2024144  2024-06-02(日)      8      1      6   816   826      1      1   
1     2024143  2024-06-01(六)      9      5      5   955     5      4      1   
2     2024142  2024-05-31(五)      8      1      3   813   990     25     14   
3     2024141  2024-05-30(四)      2      5      6   256   716      4      2   
4     2024140  2024-05-29(三)      5      3      6   536   687     11      4   
...       ...            ...    ...    ...    ...   ...   ...    ...    ...   
3925  2013038  2013-02-07(四)      6      4      0   640  None      1     -1   
3926  2013037  2013-02-06(三)      3      6      4   364  None     -1     -1   
3927  2013036  2013-02-05(二)      6      5      8   658  None     -1     -1   
3928  2013033  2013-02-02(六)      2      7      7   277  None     -1     -1   
3929  2013030  2013-01-30(三)      7      3      2   732  None     -1     -1   

      3号球遗漏  
0         2  
1         4  
2        

In [24]:
## Method 3: 找多球遗漏
def find_next_occurrence_in_columns(df, columns, value, start_index):
    for i in range(start_index + 1, len(df)):
        # 遍历每一列
        for col in columns:
            if df[col][i] == value:
                return i - start_index - 1  # 返回从当前索引到起始索引之间的距离
    return -1  # 如果没有找到匹配的值，则返回-1

columns_to_check = [COLS_BALL_1, COLS_BALL_2, COLS_BALL_3]  # 假设这是你想要检查的列名列表
COLS_BALL_1_YiLouALL = '1号球全局遗漏'
COLS_BALL_2_YiLouALL = '2号球全局遗漏'
COLS_BALL_3_YiLouALL = '3号球全局遗漏'
df[COLS_BALL_1_YiLouALL] = df.apply(
    lambda row: find_next_occurrence_in_columns(df, columns_to_check, row[COLS_BALL_1], df.index.get_loc(row.name)),
    axis=1
)
df[COLS_BALL_2_YiLouALL] = df.apply(
    lambda row: find_next_occurrence_in_columns(df, columns_to_check, row[COLS_BALL_2], df.index.get_loc(row.name)),
    axis=1
)
df[COLS_BALL_3_YiLouALL] = df.apply(
    lambda row: find_next_occurrence_in_columns(df, columns_to_check, row[COLS_BALL_3], df.index.get_loc(row.name)),
    axis=1
)
print(df)


           期号           开奖日期  开奖号码1  开奖号码2  开奖号码3  开奖号码    间隔  1号球遗漏  2号球遗漏  \
0     2024144  2024-06-02(日)      8      1      6   816   826      1      1   
1     2024143  2024-06-01(六)      9      5      5   955     5      4      1   
2     2024142  2024-05-31(五)      8      1      3   813   990     25     14   
3     2024141  2024-05-30(四)      2      5      6   256   716      4      2   
4     2024140  2024-05-29(三)      5      3      6   536   687     11      4   
...       ...            ...    ...    ...    ...   ...   ...    ...    ...   
3925  2013038  2013-02-07(四)      6      4      0   640  None      1     -1   
3926  2013037  2013-02-06(三)      3      6      4   364  None     -1     -1   
3927  2013036  2013-02-05(二)      6      5      8   658  None     -1     -1   
3928  2013033  2013-02-02(六)      2      7      7   277  None     -1     -1   
3929  2013030  2013-01-30(三)      7      3      2   732  None     -1     -1   

      3号球遗漏  1号球全局遗漏  2号球全局遗漏  3号球全局遗漏  
0         

In [25]:
df.to_csv(file_path_save, index=False)